# Imports

In [1]:
from fastai.text import *
from fastai.imports import *
from fastai.structured import *
from fastai.column_data import *
from torch.nn import functional as F
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  re

In [2]:
import gc
import pdb

# Data processing func

In [13]:
def scale_vars(df, mapper, scale_col_exc):
    warnings.filterwarnings('ignore', category=sklearn.exceptions.DataConversionWarning)
    if mapper is None:
        map_f = [([n],StandardScaler()) for n in df.columns if is_numeric_dtype(df[n]) and n not in scale_col_exc]
        mapper = DataFrameMapper(map_f).fit(df)
    df[mapper.transformed_names_] = mapper.transform(df)
    return mapper

In [14]:
def proc_df2(df, y_fld = None, skip_flds=None, do_scale=True, scale_col_exc = None, na_dict=None,
             preproc_fn=None, max_n_cat=10, subset=None, mapper=None):
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    df = df.copy()
    if preproc_fn: preproc_fn(df)
    if y_fld is not None: 
        y = df[y_fld].values
        df.drop(skip_flds+[y_fld], axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if do_scale: mapper = scale_vars(df, mapper, scale_col_exc)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    if y_fld is not None: 
        res = [pd.get_dummies(df, dummy_na=True), y, na_dict]
    else:
        res = [pd.get_dummies(df, dummy_na=True), na_dict]
    if do_scale: res = res + [mapper]
    return res

In [234]:
def df_to_maxtrix(df, cols = cols):    
    return df[cols].values.astype(np.float32)

# app train

In [17]:
df_train = pd.read_csv('data/application_train.csv')
train_cats(df_train)

In [19]:
y_label = df_train.TARGET.values
df_train.drop(columns = 'TARGET', axis=1, inplace=True)

In [20]:
df_train, _, _ = proc_df2(df_train,  do_scale=True, scale_col_exc = ['SK_ID_CURR'])

In [22]:
sk_id_curr = df_train.SK_ID_CURR.values

In [23]:
x_train,x_valid, y_train, y_valid = train_test_split(sk_id_curr, y_label, test_size=0.2, stratify = y_label)

In [229]:
cols = [i for i in df_train.columns if i not in ['SK_ID_CURR','SK_ID_PREV']]
df_train_group = df_train.groupby(['SK_ID_CURR']).apply(lambda x: df_to_maxtrix(x, cols))

# cc train

In [27]:
cc_train = pd.read_csv('data/credit_card_balance.csv')
train_cats(cc_train)

In [28]:
cc_train.drop(columns = 'SK_ID_PREV', axis=1, inplace=True)

In [29]:
cc_train, _ = proc_df2(cc_train, do_scale = False, scale_col_exc = ['SK_ID_CURR'])

In [30]:
cc_sk_id = set(cc_train['SK_ID_CURR'].values)

In [236]:
cc_train = cc_train.sort_values(by = ['SK_ID_CURR', 'MONTHS_BALANCE'])

In [237]:
cols = [i for i in cc_train.columns if i not in ['SK_ID_CURR','SK_ID_PREV']]
cc_train_group = cc_train.groupby(['SK_ID_CURR']).apply(lambda x: df_to_maxtrix(x, cols))

In [444]:
import pprint

In [448]:
cc_train_group[100006][:,:10]

array([[    -6. ,      0. , 270000. ,      0. ,      0. ,      0. ,      0. ,      0. ,   2702.7,      0. ],
       [    -5. ,      0. , 270000. ,      0. ,      0. ,      0. ,      0. ,      0. ,   2702.7,      0. ],
       [    -4. ,      0. , 270000. ,      0. ,      0. ,      0. ,      0. ,      0. ,   2702.7,      0. ],
       [    -3. ,      0. , 270000. ,      0. ,      0. ,      0. ,      0. ,      0. ,   2702.7,      0. ],
       [    -2. ,      0. , 270000. ,      0. ,      0. ,      0. ,      0. ,      0. ,   2702.7,      0. ],
       [    -1. ,      0. , 270000. ,      0. ,      0. ,      0. ,      0. ,      0. ,   2702.7,      0. ]],
      dtype=float32)

# Data loader

## function

In [358]:
class SortishSamplerRNN(Sampler):
    def __init__(self, data_source, key, cc_sk_id, bs):
        self.data_source, self.key, self.cc_sk_id, self.bs = data_source, key, cc_sk_id,bs
    
    def add_id(self, i): return i if len(i)%self.bs == 0 else i+[i[-1]]*(self.bs-len(i)%self.bs)
    
    def len_cal(self, i): 
        return self.key[self.data_source[i]].shape[0] if self.data_source[i] in self.cc_sk_id else 0

    def __len__(self): return len(self.data_source)

    def __iter__(self):        
        d={}
        for i in range(len(self.data_source)): d.setdefault(self.len_cal(i), []).append(i)        
        result = [d[n] for n in sorted(d, reverse=True)] 
        result = [self.add_id(i) for i in result]
        result = [i for j in result for i in j]
        return iter(result)

In [357]:
class ColumnarRNNDataset(Dataset):
    def __init__(self, df, y):
        n = len(df)
        self.df = df
        self.y = np.zeros((n,1)) if y is None else y

    def __len__(self): return len(self.y)

    def __getitem__(self, idx): return [self.df[idx], self.y[idx]] 

    @classmethod
    def from_data_frames(cls,df, y=None): return cls(df, y) 

    @classmethod
    def from_data_frame(cls, df, y=None): return cls.from_data_frames(df,y)

In [377]:
class ColumnarRNNModelData(ModelData):
    def __init__(self, path, trn_ds, val_ds, bs, tr_sampler = None, val_sampler = None, test_ds=None, shuffle=False):        
        trn_dl = DataLoader(trn_ds, bs, shuffle=shuffle, sampler = tr_sampler, num_workers=1) #
        val_dl = DataLoader(val_ds, bs, shuffle=shuffle, sampler = val_sampler, num_workers=1) #, 
        test_dl = DataLoader(test_ds, bs, shuffle=False, num_workers=1) if test_ds is not None else None
        super().__init__(path, trn_dl, val_dl, test_dl)

    @classmethod
    def from_data_frames(cls, path, 
                         trn_df, val_df, trn_y, val_y, 
                         tr_sampler = None, val_sampler = None,
                         bs=64, test_df=None, shuffle=False):
        trn_ds  = ColumnarRNNDataset.from_data_frame(trn_df,trn_y)
        val_ds  = ColumnarRNNDataset.from_data_frame(val_df, val_y)
        test_ds = ColumnarRNNDataset.from_data_frame(test_df, None) if test_df is not None else None
        return cls(path, 
                   trn_ds, val_ds, bs, test_ds=test_ds, 
                   tr_sampler = tr_sampler, val_sampler = val_sampler, 
                   shuffle=shuffle)

In [ ]:
class ColumnarRNNModelData2(ColumnarRNNModelData):
    @classmethod
    def from_data_frames(cls, path, 
                         trn_df, val_df, trn_y, val_y, 
                         tr_sampler = None, val_sampler = None,
                         bs=64, test_df=None, shuffle=False):
        trn_ds = 1

## build

In [400]:
bs = 64; PATH = './'

In [401]:
trn_samp = SortishSamplerRNN(x_train, cc_train_group, cc_sk_id, bs=bs)
val_samp = SortishSamplerRNN(x_valid, cc_train_group, cc_sk_id, bs=bs)

In [402]:
md  = ColumnarRNNModelData.from_data_frames(PATH, 
                                            trn_df = x_train, val_df = x_valid, 
                                            trn_y = y_train, val_y = y_valid,
                                            tr_sampler = trn_samp, val_sampler = val_samp,
                                            shuffle=False, bs=bs)

# Learner

## Class

In [2]:
class RNNModel(nn.Module):
    def __init__(self, app_ref, cc_ref, cc_sk_id, szs, app_drop, cat_drop, rnn_drop, lin_drops, bs):        
        super().__init__()
        self.bs, self.cc_sk_id = bs, cc_sk_id       
        self.app_ref, self.cc_ref = app_ref, cc_ref
        
        szs = [309] + szs
        self.rnn = nn.GRU(input_size = 37, hidden_size = 64, num_layers = 2, dropout=rnn_drop)
        
        #linear layer
        self.lins = nn.ModuleList([nn.Linear(szs[i], szs[i+1]) for i in range(len(szs)-1)])
        for o in self.lins: kaiming_normal(o.weight.data)
        self.l_outp = nn.Linear(szs[-1], 1)
        kaiming_normal(self.l_outp.weight.data)
        #batchnorm layer
        self.bns_app = nn.BatchNorm1d(245)
        self.bns_lins = nn.ModuleList([nn.BatchNorm1d(sz) for sz in szs[1:]])
        #dropout
        self.app_drop = nn.Dropout(app_drop)
        self.cat_drop = nn.Dropout(cat_drop)
        self.drops_lins = nn.ModuleList([nn.Dropout(drop) for drop in lin_drops])
        
        self.zeros = V(torch.zeros(self.bs, 1, 37))

    def forward(self, x_in):
        x_inp = x_in.cpu().data.numpy()
        
        app_input = torch.stack([V(i[0]) for i in self.app_ref[x_inp]])
        app_input = self.app_drop(self.bns_app(app_input))
        
        cc_input = self.zeros if x_inp[0] not in self.cc_sk_id else torch.stack([V(i) for i in self.cc_ref[x_inp]])        
        self.rnn.flatten_parameters()
        outp,_ = self.rnn(cc_input)
        
        x = self.cat_drop(torch.cat([app_input, outp[:,-1,:]], 1))
                                     
        for linear,drop_out,batch_norm in zip(self.lins, self.drops_lins, self.bns_lins): 
            x = drop_out(batch_norm(F.relu(linear(x))))
        x = F.sigmoid(self.l_outp(x))
        return x

In [390]:
def imbalance_loss(inp,targ):
    inp_flat = inp.view(-1); targ_flat = targ.float().view(-1)
    return F.binary_cross_entropy(inp_flat, targ_flat, targ_flat + 0.8)

## Build

In [433]:
m = RNNModel(app_ref = df_train_group, cc_ref = cc_train_group, 
             szs=  [400, 200, 100], 
             lin_drops = [0.15, 0.15, 0.25],
             app_drop = 0.15, cat_drop = 0.15, rnn_drop = 0.2,
             cc_sk_id = cc_sk_id, bs = bs)

In [434]:
RNN_learner = Learner.from_model_data(m, md)
RNN_learner.crit = imbalance_loss

## inside model

In [450]:
it = iter(md.trn_dl)

In [520]:
x, y = next(it)

In [521]:
x.cpu().numpy()

array([424713, 224283, 415461, 450171, 377005, 260304, 453933, 248617, 330204, 249270, 170463, 297895,
       333630, 183329, 238781, 327531, 207461, 168190, 387621, 128421, 248332, 208383, 117152, 131243,
       407891, 332435, 173751, 386240, 141924, 238534, 234970, 223769, 132120, 147180, 412491, 426472,
       200196, 374149, 427889, 261825, 300192, 361145, 398680, 449413, 188868, 243526, 191388, 128089,
       190101, 332167, 276902, 310027, 398777, 323515, 214678, 196818, 231733, 128713, 357860, 289818,
       369460, 444766, 173101, 154987])

In [522]:
y.cpu().numpy()

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [523]:
app_in = torch.stack([V(i[0]) for i in m.app_ref[x.cpu().numpy()]])

In [524]:
app_in

Variable containing:
 -0.5775   0.6166   0.5872  ...    1.0000   0.0000   0.0000
  0.8073   0.3319  -0.6274  ...    1.0000   0.0000   0.0000
 -0.5775  -0.2374   0.3880  ...    1.0000   0.0000   0.0000
           ...               ⋱              ...            
  0.8073   0.1421   0.6280  ...    0.0000   0.0000   1.0000
 -0.5775   0.7115   1.8407  ...    1.0000   0.0000   0.0000
 -0.5775  -0.1995  -0.8734  ...    1.0000   0.0000   0.0000
[torch.cuda.FloatTensor of size 64x245 (GPU 0)]

In [526]:
cc_input = torch.stack([V(i) for i in m.cc_ref[x.cpu().numpy()]])  

In [528]:
cc_input.size()

torch.Size([64, 96, 37])

In [529]:
cc_input[0]

Variable containing:
-9.6000e+01  8.9873e+04  1.8000e+05  ...   0.0000e+00  0.0000e+00  0.0000e+00
-9.5000e+01  8.8020e+04  1.8000e+05  ...   0.0000e+00  0.0000e+00  0.0000e+00
-9.4000e+01  8.5002e+04  1.8000e+05  ...   0.0000e+00  0.0000e+00  0.0000e+00
                ...                   ⋱                   ...                
-3.0000e+00  0.0000e+00  0.0000e+00  ...   0.0000e+00  0.0000e+00  0.0000e+00
-2.0000e+00  0.0000e+00  0.0000e+00  ...   0.0000e+00  0.0000e+00  0.0000e+00
-1.0000e+00  0.0000e+00  0.0000e+00  ...   0.0000e+00  0.0000e+00  0.0000e+00
[torch.cuda.FloatTensor of size 96x37 (GPU 0)]

In [530]:
outp,_ = m.rnn(cc_input)

In [532]:
outp.size()

torch.Size([64, 96, 64])

In [534]:
x_cat = m.cat_drop(torch.cat([app_in, outp[:,-1,:]], 1))

In [535]:
x_cat

Variable containing:
 -0.5775   0.6166   0.5872  ...   -0.2988  -0.3092  -0.5867
  0.8073   0.3319  -0.6274  ...   -0.6520  -0.6392  -0.5300
 -0.5775  -0.2374   0.3880  ...   -0.6857  -0.7685  -0.7751
           ...               ⋱              ...            
  0.8073   0.1421   0.6280  ...   -0.9324  -0.9700   0.4383
 -0.5775   0.7115   1.8407  ...   -0.9285  -0.9688   0.4304
 -0.5775  -0.1995  -0.8734  ...   -0.9332  -0.9790   0.6781
[torch.cuda.FloatTensor of size 64x309 (GPU 0)]

## fit

In [435]:
lr = 5e-2

In [438]:
RNN_learner.fit(lr, n_cycle = 5, cycle_len = 3, cycle_mult=1, use_wd_sched=True, wds=1e-5)

epoch      trn_loss   val_loss                                 
    0      0.32909    0.352099  
    1      0.323182   0.350866                                 
    2      0.325351   0.352932                                 
    3      0.327733   0.351974                                 
    4      0.329273   0.35153                                  
    5      0.328606   0.352933                                 
    6      0.325109   0.353687                                 
    7      0.323998   0.354662                                 
    8      0.327932   0.354893                                 
    9      0.320995   0.355032                                 
    10     0.320868   0.354002                                 
    11     0.327912   0.354857                                 
    12     0.321223   0.353507                                 
    13     0.3178     0.355163                                 
    14     0.336139   0.36284                                  


[array([0.36284])]